In [101]:
# %matplotlib inline

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import Combo_Functions as cf
import combo
import math 
import itertools
import sys
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


In [103]:
#Import Data
os.chdir('E:\Academics\Research\Data\Experimental data\Compression_test\Results\Input\As_Built')
Master_Data = pd.read_csv('Data_As_Built_New.csv').dropna()
len(Master_Data)

69

In [104]:
# Input and output columns in Dataframe
Input = ['X','Y','Theta','Phi','Max_Pore_Diameter']
Properties = ['Yield','Modulus']
Output = Properties[0]

In [105]:
# Normalization of data
full = []
full.extend(Input)
full.extend(Properties)
Parsed_Data = Master_Data[full]
Parsed_Data = Parsed_Data[Parsed_Data['Phi']==90]

In [106]:
IO = ['X','Y','Theta','Max_Pore_Diameter',Output]
Parsed_Data = Parsed_Data[IO]

In [107]:
new_input = ['X','Y','Theta','Max_Pore_Diameter']

In [108]:
Normalized_Data = (Parsed_Data-Parsed_Data.min())/(Parsed_Data.max()-Parsed_Data.min())
Normalized_Data = Normalized_Data.dropna()

In [109]:
# Name Input and target values
X = Normalized_Data[new_input].values
y = Normalized_Data[Output].values

In [110]:
Data = np.column_stack((X,y))

In [111]:
stdX  = np.std( Data, 0 )
index = np.where(stdX !=0)
meanX = np.mean(Data[:,index[0]],0)

In [112]:
Data = combo.misc.centering(Data)

In [113]:
L = len(Data)
b = np.arange(L)
print "Original:", b


In [114]:
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [115]:
class Iteration(object):
    pass

In [116]:
Train_R2 = []
Test_R2 = []

Train_Error = []
Test_Error = []

In [117]:
#K Fold cross validation
n = 5

In [118]:
subs = np.array(np.split(b, n))
for i in range(n):
    iteration = Iteration()
    # which step in the N-fold cross validation (which is test, which is train)
    j = range(n)
    del j[i]
    training_indices = np.concatenate(subs[j])
    testing_indices = subs[i]

    # create data subsets
    iteration.train = Data[training_indices]
    iteration.test = Data[testing_indices]

    # Train and Predict the results
    blockPrint()
    Train_predict = cf.Validate(Data[training_indices], Data[training_indices])
    Test_predict = cf.Validate(Data[training_indices],Data[testing_indices]) # test_predict
    Data_actual = Data[:,-1][testing_indices]
    Data_train = Data[:,-1][training_indices]


    # Decentering the Data
    Test_predict = Test_predict*stdX[-1]+meanX[-1]
    Train_predict = Train_predict*stdX[-1]+meanX[-1]
    Data_actual = Data_actual*stdX[-1]+meanX[-1]
    Data_train = Data_train*stdX[-1]+meanX[-1]

    # test your model
    r_test = r2_score(Data_actual,Test_predict)
    r_train = r2_score(Data_train,Train_predict)

    iteration.predict_test = Test_predict
    iteration.predict_train = Train_predict

    iteration.actual = Data_actual
    iteration.train = Data_train

    iteration.r_train = r_train
    iteration.r_test = r_test

    iteration.er_test = np.sqrt(mean_squared_error(Data_actual,Test_predict,multioutput = 'uniform_average'))
    iteration.er_train = np.sqrt(mean_squared_error(Data_train,Train_predict,multioutput = 'uniform_average'))

    # save the results (error in the testing data, error in the training data, etc.)
    iterations.append(iteration)


    

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
# plot your results
fig = plt.figure(figsize=(16,9))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

x = N
y = Train_R2
z = Test_R2

y1 = Train_Error
z1 = Test_Error

ax1.plot(x, y,'r-',label = 'Train')
ax1.plot(x,z,'b-',label = 'Test')
ax1.set_xlabel('Folds of cross validation',size = 15)
ax1.set_ylabel('R^2', size = 15)
ax1.legend(loc = 'upper right')

ax2.plot(x, y1,'r-', label = 'Train')
ax2.plot(x,z1,'b-', label = 'Test')
ax2.set_xlabel('Folds of cross validation', size = 15)
ax2.set_ylabel('Mean squared Error', size = 15)
ax2.legend(loc = 'upper right')

In [ ]:
os.chdir('E:\Academics\Research\Data\Experimental data\Compression_test\Analysis\Combo_validation')
fig.savefig('120Samples_cross-validation.jpeg')